In [1]:
%pylab inline
import scipy.stats
from collections import defaultdict  # default dictionary 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats import mode


Populating the interactive namespace from numpy and matplotlib


In [2]:
data=pd.read_csv('./iris.data')
data.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']
print (data.describe())

       SepalLength  SepalWidth  PetalLength  PetalWidth
count   149.000000  149.000000   149.000000  149.000000
mean      5.848322    3.051007     3.774497    1.205369
std       0.828594    0.433499     1.759651    0.761292
min       4.300000    2.000000     1.000000    0.100000
25%       5.100000    2.800000     1.600000    0.300000
50%       5.800000    3.000000     4.400000    1.300000
75%       6.400000    3.300000     5.100000    1.800000
max       7.900000    4.400000     6.900000    2.500000


In [3]:
# Get your data in matrix
X=np.asarray(data[['SepalLength','SepalWidth','PetalLength','PetalWidth']].dropna())
Y=np.asarray(data['Class'].dropna())
print (" Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape)  
print(X.shape)
print(Y.shape)

 Data Set Dimensions= (149, 4)  True Class labels dimensions (149,)
(149, 4)
(149,)


In [4]:
# here data is being set so that there are only two classes
Y[Y=='Iris-virginica']='Iris-versicolor'
Y[Y=='Iris-versicolor']=1
Y[Y=='Iris-setosa']=0

In [5]:
Y=Y.reshape((-1, 1))
X=np.append(X,Y,axis=1)

In [6]:
np.random.shuffle(X)
print(X)

[[6.2 3.4 5.4 2.3 1]
 [6.4 2.8 5.6 2.1 1]
 [5.0 2.3 3.3 1.0 1]
 [6.3 2.7 4.9 1.8 1]
 [6.6 3.0 4.4 1.4 1]
 [6.2 2.9 4.3 1.3 1]
 [6.3 2.5 4.9 1.5 1]
 [7.3 2.9 6.3 1.8 1]
 [4.4 3.0 1.3 0.2 0]
 [6.1 2.8 4.0 1.3 1]
 [4.4 3.2 1.3 0.2 0]
 [6.1 2.6 5.6 1.4 1]
 [7.7 2.6 6.9 2.3 1]
 [5.0 3.6 1.4 0.2 0]
 [6.0 2.9 4.5 1.5 1]
 [5.7 3.0 4.2 1.2 1]
 [5.8 2.7 3.9 1.2 1]
 [6.5 3.0 5.5 1.8 1]
 [4.9 3.1 1.5 0.1 0]
 [5.0 3.5 1.6 0.6 0]
 [5.7 2.6 3.5 1.0 1]
 [7.2 3.0 5.8 1.6 1]
 [6.9 3.2 5.7 2.3 1]
 [4.9 3.1 1.5 0.1 0]
 [6.5 3.0 5.2 2.0 1]
 [6.4 3.2 4.5 1.5 1]
 [5.3 3.7 1.5 0.2 0]
 [6.1 3.0 4.9 1.8 1]
 [4.8 3.0 1.4 0.1 0]
 [5.6 2.5 3.9 1.1 1]
 [6.8 3.0 5.5 2.1 1]
 [5.6 3.0 4.1 1.3 1]
 [5.6 2.7 4.2 1.3 1]
 [5.8 2.6 4.0 1.2 1]
 [4.8 3.0 1.4 0.3 0]
 [4.8 3.4 1.9 0.2 0]
 [6.4 3.1 5.5 1.8 1]
 [4.5 2.3 1.3 0.3 0]
 [6.0 3.0 4.8 1.8 1]
 [5.1 3.4 1.5 0.2 0]
 [6.7 3.1 5.6 2.4 1]
 [5.4 3.9 1.3 0.4 0]
 [5.8 2.8 5.1 2.4 1]
 [6.0 2.7 5.1 1.6 1]
 [7.7 3.0 6.1 2.3 1]
 [5.0 3.2 1.2 0.2 0]
 [6.0 3.4 4.5 1.6 1]
 [5.0 3.5 1.3

In [7]:
def InfoGain(Data):
    tempdata=np.zeros((Data.shape))
    f_index=value_maxinfogain=max_infogain=index_maxinfogain=maxweighted_avg=mtotaldata=uc1=lc1=muchild=mlchild=0
    setosa_inparent_prob=(Data[:,4]==0).sum()/Data.shape[0]
    versicolor_inparent_prob=(Data[:,4]==1).sum()/Data.shape[0]
    if(setosa_inparent_prob==0):
        setosa_inparent_prob=1
    if(versicolor_inparent_prob==0):
        versicolor_inparent_prob=1
   # print('Setosa prob in parent: ',setosa_inparent_prob)
   # print('Versicolor prob in parent: ',versicolor_inparent_prob)
    entropy_ofparent=-(setosa_inparent_prob*math.log(setosa_inparent_prob,2)+versicolor_inparent_prob*math.log(versicolor_inparent_prob,2))
   # print('Entropy of parent: ',entropy_ofparent,'\n')
    uchild_countsetosa=uchild_countversicolor=lchild_countsetosa=lchild_countversicolor=max_infogain=0
    uchild=np.zeros((1,5))
    lchild=np.zeros((1,5))
    min_of_Data=Data.min(axis=0)
    for feature in range(Data.shape[1]-1):
        #max_infogain=index_maxinfogain=value_maxinfogain=maxweighted_avg=mtotaldata=uc1=lc1=0
        for i in range(Data.shape[0]):
            splitpoint=Data[i,feature]#change 0 for other features
            for j in range(Data.shape[0]):
                if(Data[j,feature]<splitpoint or Data[j,feature]==min_of_Data[feature]):#change 0 for other features
                    lchild=np.vstack([lchild,Data[j,:]])
                    if(Data[j,4]==1):
                        lchild_countversicolor+=1
                    elif(Data[j,4]==0):
                        lchild_countsetosa+=1
                elif(Data[j,feature]>=splitpoint):#change 0 for other features
                    uchild=np.vstack([uchild,Data[j,:]])
                    if(Data[j,4]==1):
                        uchild_countversicolor+=1
                    elif(Data[j,4]==0):
                        uchild_countsetosa+=1
            #print('uchild_countversicolor: ',uchild_countversicolor)
            #print('uchild_countsetosa: ',uchild_countsetosa)
            #print('lchild_countversicolor: ',lchild_countversicolor)
            #print('lchild_countsetosa: ',lchild_countsetosa)
            #print('Total: ',uchild_countsetosa+uchild_countversicolor+lchild_countversicolor+lchild_countsetosa)
            uchild=np.delete(uchild,(0),axis=0)
            #print('uchild shape:',uchild.shape)
            #print('uchild:\n',uchild)
            lchild=np.delete(lchild,(0),axis=0)
            #print('lchild shape:',lchild.shape)
            #print('lchild:\n',lchild)
            uchild_setosaprob=uchild_countsetosa/(uchild_countsetosa+uchild_countversicolor)
            uchild_versicolorprob=uchild_countversicolor/(uchild_countsetosa+uchild_countversicolor)
            lchild_setosaprob=lchild_countsetosa/(lchild_countversicolor+lchild_countsetosa)
            lchild_versicolorprob=lchild_countversicolor/(lchild_countversicolor+lchild_countsetosa)
            if(uchild_setosaprob==0):
                uchild_setosaprob=1
            if(uchild_versicolorprob==0):
                uchild_versicolorprob=1
            if(lchild_setosaprob==0):
                lchild_setosaprob=1
            if(lchild_versicolorprob==0):
                lchild_versicolorprob=1
            #if(feature==2 and i==11):
            #    print('uchild_countversicolor: ',uchild_countversicolor)
            #    print('uchild_countsetosa: ',uchild_countsetosa)
            #    print('lchild_countversicolor: ',lchild_countversicolor)
            #    print('lchild_countsetosa: ',lchild_countsetosa)
            entrpoy_uchild=-(uchild_setosaprob*math.log(uchild_setosaprob,2)+uchild_versicolorprob*math.log(uchild_versicolorprob,2))
            entrpoy_lchild=-(lchild_setosaprob*math.log(lchild_setosaprob,2)+lchild_versicolorprob*math.log(lchild_versicolorprob,2))
            #print('entrpoy_uchild :',entrpoy_uchild)
            #print('entrpoy_lchild :',entrpoy_lchild)
            totaldata=uchild.shape[0]+lchild.shape[0]
            #if(feature==2 and splitpoint==3.0):
              #  print('splitpoint:',splitpoint)
               # print('uchild_countversicolor: ',uchild_countversicolor)
                #print('uchild_countsetosa: ',uchild_countsetosa)
               # print('lchild_countversicolor: ',lchild_countversicolor)
               # print('lchild_countsetosa: ',lchild_countsetosa)
               # print('uchild shape :',uchild.shape)
               # print('uchild :\n',uchild)
               # print('lchild shape :',lchild.shape)
               # print('lchild :\n',lchild)
               # print('entrpoy_uchild :',entrpoy_uchild)
               # print('entrpoy_lchild :',entrpoy_lchild)
               # print('-------------------')
            weighted_avg=(entrpoy_uchild*(uchild.shape[0]/totaldata)+entrpoy_lchild*(lchild.shape[0]/totaldata))
            info_gain=entropy_ofparent-weighted_avg
            #if(info_gain==0):
             #   print('******start******')
              #  print('\nwhere is zero index:',i)
              #  print('where is zero feature:',feature)
              #  print('where is zero splitpoint:',splitpoint)
               # print('where is zero uchild shape:',uchild.shape)
               # print('where is zero lchild shape:',lchild.shape,'\n')
                
              #  print('where is zero uchild_countsetosa:',uchild_countsetosa)
              #  print('where is zero uchild_countversicolor:',uchild_countversicolor)
              #  print('where is zero lchild_countsetosa:',lchild_countsetosa)
              #  print('where is zero lchild_countversicolor:',lchild_countversicolor,'\n')
                
              #  print('where is zero uchild_setosaprob:',uchild_setosaprob)
              #  print('where is zero uchild_versicolorprob:',uchild_versicolorprob)
              #  print('where is zero lchild_setosaprob:',lchild_setosaprob)
              #   print('where is zero lchild_versicolorprob:',lchild_versicolorprob,'\n')
                
              #  print('where is zero setosa_inparent_prob:',setosa_inparent_prob)
              #  print('where is zero versicolor_inparent_prob:',versicolor_inparent_prob)
              #  print('where is zero entropy_ofparent',entropy_ofparent,'\n')
                
              #  print('where is zero entrpoy_uchild:',entrpoy_uchild)
              #  print('where is zero entrpoy_lchild:',entrpoy_lchild)
              #  print('where is zero weighted_avg:',weighted_avg,'\n')
              #  print('------end--------')
            if(info_gain>max_infogain):
                max_infogain=info_gain
                index_maxinfogain=i
                value_maxinfogain=splitpoint
                maxweighted_avg=weighted_avg
                #uc1=entrpoy_uchild
                #lc1=entrpoy_lchild
                #mtotaldata=totaldata
                muchild=uchild
                mlchild=lchild
                f_index=feature
            #print('For Index: ',i,'Data: ',Data[i,0])
            #if(feature==3):
               # print('For Index: ',i,'Data: ',Data[i,feature])
               # print('INFO GAIN: ',info_gain,'\n')
            uchild=np.zeros((1,5))
            lchild=np.zeros((1,5))
            uchild_countsetosa=uchild_countversicolor=lchild_countsetosa=lchild_countversicolor=0
    return value_maxinfogain,max_infogain,mlchild,muchild,f_index,maxweighted_avg

In [8]:
class Node:
    def __init__(self,purity,klasslabel='',score=0,split=[],fidx=-1):
        self.lchild=None       
        self.rchild=None
        self.klasslabel=klasslabel        
        self.split=split
        self.score=score
        self.fidx=fidx
        self.purity=purity
        
    def set_childs(self,lchild,rchild):
        # YOUR CODE HERE
        self.lchild=lchild
        self.rchild=rchild
        
    def isleaf(self):
        # YOUR CODE HERE
        if self.lchild==None and self.rchild==None:
            return True
        else:
            return False
        
        raise NotImplementedError()
    def isless_than_eq(self, X):
        # YOUR CODE HERE
        raise NotImplementedError()
    def get_str(self):        
        if self.isleaf():
            return 'C(class={},Purity={})'.format(self.klasslabel,self.purity)
        else:
            return 'I(Fidx={},Score={},Split={})'.format(self.fidx,self.score,self.split)
    

In [9]:
class DecisionTree:
    ''' Implements the Decision Tree For Classification... '''
    def __init__(self, purityp, exthreshold,maxdepth=10):        
        self.purity=purityp
        self.exthreshold=exthreshold
        self.maxdepth=maxdepth
        pass
    def train(self, X, Y):
        ''' Train Decision Tree using the given 
            X [m x d] data matrix and Y labels matrix
            
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns:
            -----------
            Nothing
           
        '''
        c1_prob=(X[:,4]==0).sum()/X.shape[0]
        c2_prob=(X[:,4]==1).sum()/X.shape[0]
        if(c1_prob==0):
            c1_prob=1
        if(c2_prob==0):
            c2_prob=1
        entropy_ofparent=-(c1_prob*math.log(c1_prob,2)+c2_prob*math.log(c2_prob,2))
        nexamples,nfeatures=X.shape
        ## now go and train a model for each class...
        # YOUR CODE HERE
        depth=0   
        self.tree = self.build_tree(X,Y,0)
        
        
    def build_tree(self, X, Y, depth):
        # YOUR CODE HERE
        #compare purity
        #infogain call gain
        #node.lchild(lchild)
        #node.uchild(uchild)
        c1_prob=(X[:,4]==0).sum()/X.shape[0]
        c2_prob=(X[:,4]==1).sum()/X.shape[0]
        checkpurity=0
        if(c1_prob>c2_prob):
            checkpurity=c1_prob
            label=0
        else:
            checkpurity=c2_prob
            label=1
        node=Node(checkpurity)
        print('888888888888888888888: ',checkpurity)
        print('999999999999999999999: ',self.purity)
        if(checkpurity>=self.purity): 
            node.klasslabel=label
            return node
        data,nfeatures=X.shape
        if(data<=self.exthreshold or depth==self.maxdepth+1):
            node.klasslabel=label
            return node
        splitpoint,infogain,Lchild,Uchild,f_index,maxweighted_avg=InfoGain(X)
        node.split=splitpoint
        #self.klasslabel=label        
        self.score=maxweighted_avg
        self.fidx=f_index
        
      #  print('Wanna see int:\n',Lchild)
       # print(type(Lchild))
       # print(Lchild.shape)
        left_targets=Lchild[:,4].copy()
        #left_targets=np.zeros((Lchild[:,4].shape))
        #left_targets=Lchild[:,4]
        left_targets=left_targets.reshape((-1, 1))
      #  print(left_targets.shape)
      #  print(left_targets)
        
        right_targets=Uchild[:,4].copy()
        #right_targets=np.zeros((Uchild[:,4].shape))
        #right_targets=Uchild[:,4]
        right_targets=right_targets.reshape((-1, 1))
     #   print(right_targets.shape)
      #  print(right_targets)
        node.rchild=self.build_tree(Uchild,right_targets,depth+1)
        node.lchild=self.build_tree(Lchild,left_targets,depth+1)
        
        return node
        print (node.split,node.fidx,node.lchild,node.rchild)
        
    def test(self, X):
        
        ''' Test the trained classifiers on the given set of examples 
        
                   
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional test examples.
           
            Returns:
            -----------
                pclass: the predicted class for each example, i.e. to which it belongs
        '''
        X=np.delete(X,(0),axis=1)
        pclasses=[]
        nexamples, nfeatures=X.shape
        
        for i in range(0,nexamples):
            pclasses.append(self.predict(X[i]))
        return pclasses
    
        
        #pass
           
    def predict(self, X):
        node=self.tree
        while (not node.isleaf()):
            if(X[node.fidx] == node.fidx):
                node=node.lchild
            else:
                node=node.rchild
        return node.klasslabel
        #pass
    
    def _predict(self,node, X):
   
        raise NotImplementedError()

    def __str__(self):
        
        return self.__print(self.tree)        
        
     
    def find_depth(self):
        return self._find_depth(self.tree)
    
    
    def _find_depth(self,node):
        if not node:
            return
        if node.isleaf():
            return 1
        else:
            return max(self._find_depth(node.lchild),self._find_depth(node.rchild))+1
    def __print(self,node,depth=0):
        
        ret = ""

        # Print right branch
        if node.rchild:
            ret += self.__print(node.rchild,depth+1)

        # Print own value
        
        ret += "\n" + ("    "*depth) + node.get_str()

        # Print left branch
        if node.lchild:
            ret += self.__print(node.lchild,depth+1)

In [10]:
Xtrain=X[0:120]
Ytrain=Y[0:120]
splitpoint,infogain,lchild,uchild,f_index,maxweighted_avg=InfoGain(X)
#print('Splitpoint: ',splitpoint)
#print('Max INFO GAIN: ',infogain)
#print('Feature index: ',f_index)
#print('Weighted averge: ',maxweighted_avg)
#print('lchild :',lchild.shape)
#print('lchild :\n',lchild)
#print('uchild :',uchild.shape)
#print('uchild :\n',uchild)

In [11]:
# Split your data into training and test-set...
Y=Y.reshape((-1, 1))
Xtrain=X[0:120]
Ytrain=Y[0:120]
Xtest=X[120:149]
Ytest=Y[120:149]
print (" Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape)   
print (" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytest.shape)
#we=np.array(X[:,4])
#we=we.reshape((-1, 1))
#print(we)

 Training Data Set Dimensions= (120, 5) Training True Class labels dimensions (120, 1)
 Test Data Set Dimensions= (29, 5) Test True Class labels dimensions (29, 1)


In [12]:
# Lets train a Decision Tree Classifier on Petal Length and Width
#feat=[0,1]
#dt=DecisionTree(0.95,5)
#dt.train(Xtrain[:,feat],Ytrain)
#print (dt)

In [13]:
#Lets test it on the set of unseen examples...
#feat=[0,1]
#pclasses=dt.test(Xtest[:,feat])

In [14]:
#Lets Train on all four features....

# Lets train a Decision Tree Classifier on Petal Length and Width

feat=[0, 1, 2, 3]
dt=DecisionTree(0.95,5)
dt.train(Xtrain,Ytrain)
#print(dt)
#print(Xtest[:,4])
#print(Xtest)
Ytest=Xtest[:,4].copy()
print('check',Ytest)
pclasses=dt.test(Xtest)
#print(pclasses)
#Lets see how good we are doing, by finding the accuracy on the test set..
print (np.sum(pclasses==Ytest))
print ("Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0]))

888888888888888888888:  0.6583333333333333
999999999999999999999:  0.95
888888888888888888888:  1.0
999999999999999999999:  0.95
888888888888888888888:  1.0
999999999999999999999:  0.95
check [0 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0]
21
Accuracy =  0.7241379310344828
